# Chapter 06: LoRA를 활용한 SFT

## 1. 학습 목표

* LoRA(Low-Rank Adaptation)의 핵심 파라미터를 이해하고 설정한다.
* `SFTTrainer`를 사용하여 데이터셋과 모델을 연결하고 학습 루프를 실행한다.
* 학습된 어댑터(Adapter)를 저장하고, 로드하여 추론 테스트를 수행한다.
* 학습 손실(Loss) 그래프를 통해 학습이 정상적으로 진행되는지 확인한다.

## 2. LoRA 이론 복습

### 2.1 LoRA의 핵심 원리

거대 언어 모델의 모든 파라미터를 학습하는 대신, 가중치 행렬의 변화량()을 저순위 행렬의 곱()으로 근사하여 학습하는 방식이다.

* **메모리 효율**: 전체 파라미터의 1% 미만만 학습하므로 VRAM 사용량이 획기적으로 줄어든다.
* **모듈성**: 학습된 어댑터 용량이 매우 작아(수백 MB), 하나의 베이스 모델에 여러 어댑터를 갈아끼우며 사용할 수 있다.

### 2.2 주요 하이퍼파라미터

| 파라미터 | 설명 | 권장값 (20B 모델 기준) |
| --- | --- | --- |
| **r (Rank)** | 저순위 행렬의 차원. 높을수록 표현력 증가하지만 메모리 사용량도 늘어난다. | 16 ~ 64 |
| **lora_alpha** | 학습 업데이트의 스케일링 팩터. 보통 Rank의 2배로 설정한다. | 32 ~ 128 |
| **target_modules** | LoRA를 적용할 레이어. Attention과 FFN 모듈 전체에 적용하는 추세다. | `["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]` |
| **lora_dropout** | 과적합 방지를 위한 드롭아웃 비율. | 0.05 ~ 0.1 |

## 3. 실습 환경 준비 및 라이브러리 임포트

필요한 라이브러리를 불러오고, 학습에 사용할 디바이스 상태를 확인한다.

In [13]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset
import os

# CUDA 사용 가능 여부 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"사용 디바이스: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

사용 디바이스: cuda
GPU: NVIDIA H100 80GB HBM3


## 4. 데이터셋 준비 및 전처리

Chapter 03에서 다룬 것과 같이 데이터셋을 로드하고, 모델이 이해할 수 있는 프롬프트 형식으로 변환한다. 여기서는 `tatsu-lab/alpaca` 데이터셋을 사용한다.

In [14]:
# 1. 데이터셋 로드
# 실습을 위해 1,000개 샘플만 사용한다. (실제 학습 시에는 전체 데이터 사용)
dataset = load_dataset("tatsu-lab/alpaca", split="train[:1000]")

# 2. 프롬프트 포맷팅 함수 정의
def format_instruction(example):
    """
    Alpaca 형식의 데이터를 LLM 학습을 위한 텍스트 프롬프트로 변환한다.
    """
    instruction = example['instruction']
    input_text = example.get('input', '')
    output = example['output']

    # 입력(Input) 유무에 따른 포맷 분기
    if input_text:
        text = f"""### Instruction:
{instruction}

### Input:
{input_text}

### Response:
{output}"""
    else:
        text = f"""### Instruction:
{instruction}

### Response:
{output}"""

    return {"text": text}

# 3. 데이터셋 매핑
formatted_dataset = dataset.map(format_instruction)

print(f"데이터셋 준비 완료: {len(formatted_dataset)}개")
print("\n[샘플 데이터]")
print(formatted_dataset[0]['text'])

데이터셋 준비 완료: 1000개

[샘플 데이터]
### Instruction:
Give three tips for staying healthy.

### Response:
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule.


## 5. 모델 및 토크나이저 로드 (QLoRA)

Chapter 05에서 실습한 대로 `gpt-oss-20b`과 별행으로 `Qwen/Qwen3-14B` 모델을 4-bit로 로드한다.
단 `gpt-oss-20b`는 4bit로 이미 양자화되어 학습한 모델임으로 로드에 주의하자.

In [15]:
# 모델 ID 설정 
model_id = "Qwen/Qwen3-14B"

# # 1. 4-bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# 2. 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="sdpa"  # 최신 PyTorch 사용 시 가속
)

# 3. 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # trl의 SFTTrainer는 right padding을 지원한다.

# 4. 학습 전처리 (Gradient Checkpointing 활성화 등)
model = prepare_model_for_kbit_training(model)

print("모델 및 토크나이저 로드 완료")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

모델 및 토크나이저 로드 완료


### 일반 16bit 모델과 4bit 모델 로드를 위한 사용자 정의함수

In [16]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# optional imports (환경에 따라 없을 수 있음)
try:
    from transformers import BitsAndBytesConfig
except Exception:
    BitsAndBytesConfig = None

try:
    from transformers import Mxfp4Config
except Exception:
    Mxfp4Config = None

try:
    from peft import prepare_model_for_kbit_training
except Exception:
    prepare_model_for_kbit_training = None


def _is_gpt_oss(model_id: str) -> bool:
    mid = (model_id or "").lower()
    return "openai/gpt-oss" in mid or mid.startswith("openai/gpt-oss")


def _make_bnb_nf4_4bit_config():
    if BitsAndBytesConfig is None:
        raise ImportError(
            "BitsAndBytesConfig를 import할 수 없음. "
            "pip install -U bitsandbytes transformers accelerate 를 확인하라."
        )
    return BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

def load_model_and_tokenizer(
    model_id: str,
    *,
    device_map: str = "auto",
    trust_remote_code: bool = True,
    gpt_oss_dequantize_to_bf16: bool = False,  # gpt-oss-20b만 해당
    enable_gradient_checkpointing: bool = True,
):
    """
    - gpt-oss-20b: MXFP4 네이티브 로드(기본) 또는 BF16 디양자화 로드
    - 그 외: bitsandbytes NF4 4bit 로드 + (가능하면) k-bit 학습 준비
    """
    is_oss = _is_gpt_oss(model_id)

    if is_oss:
        if Mxfp4Config is None:
            raise ImportError(
                "Mxfp4Config를 import할 수 없음. "
                "pip install -U transformers 로 업데이트하라."
            )

        quant_cfg = Mxfp4Config(dequantize=bool(gpt_oss_dequantize_to_bf16))
        torch_dtype = torch.bfloat16 if gpt_oss_dequantize_to_bf16 else "auto"

        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map=device_map,
            trust_remote_code=trust_remote_code,
            quantization_config=quant_cfg,
            torch_dtype=torch_dtype,
            # MXFP4 환경에서 sdpa가 안 맞는 경우가 있어 안전하게 eager 권장
            attn_implementation="eager",
        )

        mode = "gpt-oss / MXFP4"
        if gpt_oss_dequantize_to_bf16:
            mode += " (dequantize->bf16)"

    else:
        bnb_cfg = _make_bnb_nf4_4bit_config()

        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map=device_map,
            trust_remote_code=trust_remote_code,
            quantization_config=bnb_cfg,
            torch_dtype=torch.bfloat16,
            attn_implementation="sdpa",
        )

        # QLoRA/4bit 학습 준비(가능할 때만)
        if prepare_model_for_kbit_training is not None:
            model = prepare_model_for_kbit_training(model)
        mode = "bnb / NF4 4bit (QLoRA-ready)"

    # 공통: 학습 준비 옵션(권장)
    if enable_gradient_checkpointing and hasattr(model, "gradient_checkpointing_enable"):
        model.gradient_checkpointing_enable()
    if hasattr(model.config, "use_cache"):
        model.config.use_cache = False  # 학습 시 권장

    # tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=trust_remote_code)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer, {"mode": mode}

In [17]:
# gpt-oss-20b 로드 시
model_id = "openai/gpt-oss-20b"
# model_id = "Qwen/Qwen3-14B"

# 1. gpt-oss-20b: MXFP4 네이티브 4bit
model, tokenizer, info = load_model_and_tokenizer(
    model_id, 
    gpt_oss_dequantize_to_bf16=False
)

# 2. gpt-oss-20b: BF16로 디양자화(학습 파이프라인 맞출 때)
# model, tokenizer, info = load_model_and_tokenizer(model_id, gpt_oss_dequantize_to_bf16=True)

print(info["mode"])
print("model dtype:", getattr(model, "dtype", None))

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

gpt-oss / MXFP4
model dtype: torch.bfloat16


## 6. LoRA 어댑터 설정 및 적용

베이스 모델에 LoRA 어댑터를 부착한다. 모델의 모든 선형 레이어(`Linear`)를 타겟으로 지정하면 성능이 극대화된다.

In [18]:
# LoRA 설정
lora_config = LoraConfig(
    r=16,                       # Rank: 16 (일반적인 권장값)
    lora_alpha=32,              # Alpha: 32 (Rank의 2배)
    target_modules=[            # 모든 Linear 레이어에 적용
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.05,          # 드롭아웃
    bias="none",                # 바이어스 미학습
    task_type=TaskType.CAUSAL_LM
)

# 모델에 LoRA 적용
model = get_peft_model(model, lora_config)

# 학습 가능한 파라미터 수 확인
model.print_trainable_parameters()

trainable params: 7,962,624 || all params: 20,922,719,808 || trainable%: 0.0381


## 7. SFTTrainer 설정 및 학습 실행

`trl` 라이브러리의 `SFTTrainer`를 사용하여 학습을 설정하고 실행한다. 메모리 부족(OOM)을 방지하기 위해 배치 크기와 그라디언트 누적(Accumulation)을 적절히 조절해야 한다.

In [19]:
# 학습 결과 저장 경로
output_dir = "./GPT-OSS-20B-SFT-LoRA"

# 학습 설정 (SFTConfig)
training_args = SFTConfig(
    output_dir=output_dir,

    # 학습 하이퍼파라미터
    num_train_epochs=3,                 # 에포크 수 (실습용 1, 실제 3 권장)
    per_device_train_batch_size=4,      # GPU당 배치 크기 (메모리에 따라 조절)
    gradient_accumulation_steps=4,      # 그라디언트 누적 (실효 배치 크기 = 4 * 4 = 16)
    learning_rate=2e-4,                 # QLoRA 권장 학습률

    # 최적화 및 스케줄러
    optim="paged_adamw_8bit",           # 메모리 절약형 옵티마이저
    lr_scheduler_type="cosine",         # 코사인 스케줄러
    warmup_ratio=0.03,                  # 워밍업 비율

    # 정밀도 및 로깅
    fp16=False,
    bf16=True,                          # Ampere(30/40번대) GPU 이상 권장
    logging_steps=10,                   # 10 스텝마다 로그 출력

    # 저장 전략
    save_strategy="steps",
    save_steps=50,                      # 50 스텝마다 체크포인트 저장

    # 데이터 처리
    #max_seq_length=512,                 # 시퀀스 길이 제한 (메모리 절약)
    dataset_text_field="text",          # 데이터셋의 텍스트 필드명
    packing=False,                      # 데이터 패킹 비활성화 (개별 학습)

    report_to="none"                    # W&B 사용 시 "wandb"로 변경
)

# Trainer 생성
trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    args=training_args,
    processing_class=tokenizer,
    peft_config=lora_config             # LoRA 설정 전달
)

print("학습 시작...")
trainer.train()
print("학습 완료!")

/home/josh/anaconda3/envs/llm/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': 199998}.


학습 시작...


Step,Training Loss
10,2.441700
20,1.564200
30,1.378300
40,1.359900
50,1.415800
60,1.336600
70,1.350200
80,1.289400
90,1.302600
100,1.330000


학습 완료!


## 8. 모델 저장 및 테스트

학습이 완료되면 어댑터(LoRA 가중치)를 저장하고, 간단한 추론 테스트를 통해 성능을 검증한다.

In [20]:
# 1. 최종 모델(어댑터) 저장
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"모델 저장 완료: {output_dir}")

# 2. 추론 테스트 함수
def generate_response(prompt, model, tokenizer):
    """
    학습된 모델로 응답을 생성하는 함수다.
    """
    formatted_prompt = f"### Instruction:\n{prompt}\n\n### Response:\n"

    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # 프롬프트 부분 제거하고 응답만 추출
    return response.split("### Response:\n")[-1].strip()

# 3. 테스트 실행
test_prompts = [
    "인공지능의 미래에 대해 설명해줘.",
    "Python으로 피보나치 수열을 구하는 코드를 작성해줘."
]

print("\n[추론 테스트 결과]")
for prompt in test_prompts:
    print("-" * 50)
    print(f"질문: {prompt}")
    response = generate_response(prompt, model, tokenizer)
    print(f"답변: {response}")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
Caching is incompatible with gradient checkpointing in GptOssDecoderLayer. Setting `past_key_values=None`.


모델 저장 완료: ./GPT-OSS-20B-SFT-LoRA

[추론 테스트 결과]
--------------------------------------------------
질문: 인공지능의 미래에 대해 설명해줘.


/home/josh/anaconda3/envs/llm/lib/python3.11/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


답변: 인(10. **Instruction() {
                // Create a 'value': config = 2002]` has the component is `Array} \) { 
        # In PHP code to manage their responses
    def _instance`. The 'B be computed earlier text string manipulation of the 'string.IsDefined Functions

- **Label>3, 'click(function () => "What can solve this code snippet does not found on a vector<int64-bit number n^3/4) + dα+1, 'https://cdn.icons.fifoBuffer(buf: The output file. So they might be more information for an object
--------------------------------------------------
질문: Python으로 피보나치 수열을 구하는 코드를 작성해줘.
답변: def def _context` is a 'label> output: A. So the code snippet above text, or 'reactive behavior of the number 2+1e5/3- `False

The 'A$ = \(\math.pi / 4 => 1
    print(f" button = 100, "isNaN} \) + z + 0;
            int(data[1, they might be used for file and the `get_data.data.data.data.decode('Password(password, perhaps a given description, I can use a function, and the system's inventory, the code is se

## 9. 요약

이 챕터에서는 **LoRA를 활용한 SFT**의 전체 파이프라인을 실습했다.

1. **데이터 준비**: Alpaca 포맷 데이터를 모델 학습용 프롬프트로 변환했다.
2. **QLoRA 로딩**: 4-bit 양자화된 베이스 모델을 로드하여 메모리 효율을 확보했다.
3. **LoRA 설정**: 모든 선형 레이어에 어댑터를 부착하여 학습 성능을 극대화했다.
4. **학습 실행**: `SFTTrainer`와 `paged_adamw_8bit` 옵티마이저를 통해 안정적으로 학습을 수행했다.

이로써 기본적인 파인튜닝 과정은 마쳤다. 다음 챕터에서는 QLoRA에 특화된 심화 내용과 팁을 더 깊이 다룬다.

다음 챕터는 **Chapter 07: QLoRA를 활용한 SFT**로, 이미 6장에서 QLoRA를 사용했지만, 7장에서는 QLoRA의 내부 동작 원리와 메모리 최적화에 대해 더 심도 있게 다루거나 6장과 통합하여 진행할 수 있다. (사용자의 요청 순서에 따라 7장을 진행한다.)